In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from typing import Sequence, Callable
from typing import Concatenate
import subprocess

In [2]:
class Point(ABC):
    
    def __init__(self, arr: Sequence) -> None:
        self._correct_seq(arr)
        self.array = np.array(arr)
        
    def norm(self) -> 'Point':
        return self.__class__(self.array / np.linalg.norm(self.array))
        
    def get_mod(self) -> float:
        return np.linalg.norm(self.array)
        
    @abstractmethod
    def _correct_seq(self, arr: Sequence) -> None:
        raise NotImplementedError
    
    def get(self, index: int) -> float:
        return self.array[index]
    
    def __getitem__(self, index):
        return self.array[index]
    
    def __setitem__(self, index, value):
        self.array[index] = value
        
    def __add__(self, other):
        new_array = self.array + other.array
        return self.__class__(new_array)  # Создаем новый экземпляр того же класса с новым массивом
    
    def __mul__(self, other):
        new_array = other * self.array
        return self.__class__(new_array)  # Создаем новый экземпляр того же класса с новым массивом
    
    def __rmul__(self, other):
        new_array = other * self.array
        return self.__class__(new_array)

    def __sub__(self, other):
        new_array = self.array - other.array
        return self.__class__(new_array)  # Создаем новый экземпляр того же класса с новым массивом
    
    def __neg__(self):
        new_array = -self.array  # Применяем унарный минус к массиву точек
        return self.__class__(new_array)

In [3]:



class Point7d(Point):
    
    def _correct_seq(self, arr: Sequence) -> None:
        if len(arr) != 7:
            raise ValueError(f"Размер массива = {len(arr)} не соответсвует ошиданию = {7}")

In [4]:
class Point1d(Point):
    
    def _correct_seq(self, arr: Sequence) -> None:
        if len(arr) != 1:
            raise ValueError(f"Размер массива = {len(arr)} не соответсвует ошиданию = {1}")

In [5]:
class Point2d(Point):
    
    def _correct_seq(self, arr: Sequence) -> None:
        if len(arr) != 2:
            raise ValueError(f"Размер массива = {len(arr)} не соответсвует ошиданию = {2}")

In [6]:

class Func(ABC):
        
    def get_grad(self, point: Point) -> Point:
        raise NotImplementedError
    
    @abstractmethod
    def create_point(self, arr: Sequence) -> Point:
        raise NotImplementedError
    
    @abstractmethod
    def size(self) -> int:
        raise NotImplementedError

    @abstractmethod
    def __call__(self, point: Point) -> float:
        raise NotImplementedError

In [7]:

class Func2d(Func):
    
    
    def get_grad(self, point: Point2d) -> Point2d:
        x1, x2 = point[0], point[1]
        return Point2d([4*x1-x2-1, 2*x2 - x1 - 1])
    
    def create_point(self, arr: Sequence) -> Point7d:
        return Point2d(arr)
    
    def size(self) -> int:
        return 2
    
    def __call__(self, point: Point2d) -> float:
        x1, x2 = point[0], point[1]
        return 2*x1**2 - x1*x2 + x2**2 - x1 - x2 + 1

In [8]:

class Func7d(Func):
    
    def create_point(self, arr: Sequence) -> Point7d:
        return Point7d(arr)
    
    def size(self) -> int:
        return 7
    
    def __call__(self, point: Point7d) -> float:
        return np.abs(np.sum(point.array))
        #return point[0] + point[1] * point[2] - point[3] / point[4] + point[5] * point[6]
    

In [9]:
class Func1d(Func):
    
    def create_point(self, arr: Sequence) -> Point7d:
        return Point1d(arr)
    
    def size(self) -> int:
        return 1
    
    def __call__(self, point: Point1d) -> float:
        return point[0] ** 2
        return point[0]**6 + point[0]**4 - 6 * point[0]**3 + point[0]**5 + point[0]**2 + point[0]
    

In [10]:
def is_ok(point: Point) -> bool:
    return True

In [11]:
def random_points(func: Func7d, count_points: int) -> Point7d:
    points = [
        Point7d(np.random.uniform(low=-10, high=10, size=func.size()))
        for _ in range(count_points)
    ]
    
    filtered_points = list(filter(lambda p: is_ok(p), points))
    
    if not filtered_points:
        raise ValueError("Нет подходящих точек после фильтрации")
    
    return min(filtered_points, key=lambda p: func(p))

min_point = random_points(Func7d(), 1000)
print("Минимальная точка:", min_point.array)


Минимальная точка: [-4.67852847  3.42444391  0.52491785  2.11692065  2.8349823  -1.44872505
 -2.7837421 ]


In [12]:
def random_points(func: Func, count_points: int) -> tuple[Point7d, pd.DataFrame]:
    points = [
        func.create_point(np.random.uniform(low=-10, high=10, size=func.size()))
        for _ in range(count_points)
    ]
    df = pd.DataFrame([point.array for point in points])
    
    filtered_points = list(filter(lambda p: is_ok(p), points))
    
    if not filtered_points:
        raise ValueError("Нет подходящих точек после фильтрации")
    
    return min(filtered_points, key=lambda p: func(p)), df

min_point, df = random_points(Func7d(), 1000)
print("Минимальная точка:", min_point.array)
print(df)

Минимальная точка: [ 3.67821378 -9.74243856  8.84425145 -6.13514687  3.97827662 -1.91911212
  1.29908653]
            0         1         2         3         4         5         6
0    8.876048 -4.515627  9.370458  4.511626  0.302855 -6.210540  9.784387
1    2.723404  1.207800  4.598126 -6.521045 -0.740136 -5.759993 -7.085861
2   -5.508054 -1.807405  9.129087  1.010539  4.458540  2.173471 -5.582520
3   -0.695591 -1.177233 -3.087054 -7.651959 -7.401277  9.078080 -6.478383
4    5.031653  6.274529 -3.131942  5.602270  6.556582  1.690060 -1.484077
..        ...       ...       ...       ...       ...       ...       ...
995 -2.110009 -9.895669 -3.483970 -7.178203  4.612533  9.474136 -2.400655
996  8.235607 -5.978914 -3.791180  1.102479  9.809611 -2.040646 -5.007551
997  2.584196  5.870127  5.263881  5.413396  0.446130 -4.482704 -2.710527
998  3.937029 -4.659708 -0.339539 -4.480854 -7.663597 -9.063141 -7.490519
999 -6.470990  2.801154 -7.272240  9.035433 -2.445893 -8.301115  4.196357

[1000

In [13]:
min_point, df = random_points(Func1d(), 100)
print("Минимальная точка:", min_point.array)
print(df)

Минимальная точка: [-0.24980857]
           0
0  -5.129760
1   0.922724
2  -7.596457
3  -4.499333
4   8.372658
..       ...
95 -6.675206
96 -7.143180
97 -7.641696
98 -1.168184
99  1.445596

[100 rows x 1 columns]


In [14]:
def random_direction(func: Func, eps: float=0.005, step: float=0.1, count_failures: int=1):
    def create_random_vec(size) -> np.ndarray:
        random_vector = np.random.randn(size)
        return random_vector / np.linalg.norm(random_vector)

    random_vector = create_random_vec(func.size())

    point = func.create_point(np.random.rand(func.size()))
    point_new = func.create_point(point.array + step * random_vector)

    f_point = func(point)
    f_new_point = func(point_new)

    count_fails = 0
    while abs(f_point - f_new_point)/2 > eps:
        point = point_new
        point_new = func.create_point(point.array + step * random_vector)
        f_point = func(point)
        f_new_point = func(point_new)
        if f_new_point > f_point:
            count_fails += 1
        else:
            count_fails = 0
            
        if count_fails >= count_failures:
            point = func.create_point(point.array - count_fails * random_vector)
            random_vector = create_random_vec(func.size())
            point_new = func.create_point(point.array + step * random_vector)
            count_fails = 0

    return point_new



In [15]:
point = random_direction(Func1d())
print(point.array)
print(Func1d()(point))

[0.00293454]
8.611539025251811e-06


In [16]:
point = random_direction(Func7d())
print(point.array)
print(Func7d()(point))

[-2.55791253  3.5401885  -2.70149346  3.03542017 -1.14338811 -4.28351095
  4.50247843]
0.39178205505379093


In [17]:
def random_direction(func: Func, eps: float=0.005, step: float=0.1, count_failures: int=1) -> tuple[Point, pd.DataFrame]:
    def create_random_vec(size) -> np.ndarray:
        random_vector = np.random.randn(size)
        return random_vector / np.linalg.norm(random_vector)
    
    df = []

    random_vector = create_random_vec(func.size())

    point = func.create_point(np.random.rand(func.size()))
    df.append(point.array)
    point_new = func.create_point(point.array + step * random_vector)

    f_point = func(point)
    f_new_point = func(point_new)

    count_fails = 0
    while abs(f_point - f_new_point)/2 > eps:
        point = point_new
        df.append(point.array)
        point_new = func.create_point(point.array + step * random_vector)
        f_point = func(point)
        f_new_point = func(point_new)
        if f_new_point > f_point:
            count_fails += 1
        else:
            count_fails = 0
            
        if count_fails >= count_failures:
            point = func.create_point(point.array - count_fails * random_vector)
            df.append(point.array)
            random_vector = create_random_vec(func.size())
            point_new = func.create_point(point.array + step * random_vector)
            count_fails = 0

    return point_new, pd.DataFrame(df)

In [18]:
point, df = random_direction(Func2d())
print(point.array)
print(df)
print(Func2d()(point))
df.to_csv('my_data.csv', index=False)

[0.60134239 0.56930506]
          0         1
0  0.455214  0.307300
1  0.503923  0.394635
2  0.552633  0.481970
0.5343388735640373


In [19]:
def random_direction_correcting_steps(func: Func, eps: float=0.005, step: float=2, count_failures: int=3):
    def create_random_vec(size) -> np.ndarray:
        random_vector = np.random.randn(size)
        return random_vector / np.linalg.norm(random_vector)

    random_vector = create_random_vec(func.size())

    point = func.create_point(np.random.rand(func.size()))
    point_new = func.create_point(point.array + step * random_vector)

    f_point = func(point)
    f_new_point = func(point_new)

    count_fails = 0
    while abs(f_point - f_new_point)/2 > eps:
        point = point_new
        point_new = func.create_point(point.array + step * random_vector)
        f_point = func(point)
        f_new_point = func(point_new)
        if f_new_point > f_point:
            count_fails += 1
            step /= 2
        else:
            count_fails = 0
            
        if count_fails >= count_failures:
            point = func.create_point(point.array - count_fails * random_vector)
            random_vector = create_random_vec(func.size())
            point_new = func.create_point(point.array + step * random_vector)
            count_fails = 0

    return point_new

In [20]:
point = random_direction_correcting_steps(Func1d())
print(point.array)
print(Func1d()(point))

[0.14191624]
0.020140219427659437


In [21]:
point = random_direction_correcting_steps(Func7d())
print(point.array)
print(Func7d()(point))

[-1.70212937  1.00716508 -0.17316866 -0.20288286  0.02042071  0.06871041
  1.22068452]
0.23879983031396756


In [22]:
def random_direction_correcting_steps(func: Func, eps: float=0.005, step: float=2, count_failures: int=3) -> tuple[Point, pd.DataFrame]:
    def create_random_vec(size) -> np.ndarray:
        random_vector = np.random.randn(size)
        return random_vector / np.linalg.norm(random_vector)

    random_vector = create_random_vec(func.size())

    df = []
    point = func.create_point(np.random.rand(func.size()))
    df.append(point.array)
    point_new = func.create_point(point.array + step * random_vector)

    f_point = func(point)
    f_new_point = func(point_new)

    count_fails = 0
    while abs(f_point - f_new_point)/2 > eps:
        point = point_new
        df.append(point.array)
        point_new = func.create_point(point.array + step * random_vector)
        f_point = func(point)
        f_new_point = func(point_new)
        if f_new_point > f_point:
            count_fails += 1
            step /= 2
        else:
            count_fails = 0
            
        if count_fails >= count_failures:
            point = func.create_point(point.array - count_fails * random_vector)
            df.append(point.array)
            random_vector = create_random_vec(func.size())
            point_new = func.create_point(point.array + step * random_vector)
            count_fails = 0

    return point_new, pd.DataFrame(df)

In [23]:
point, df = random_direction_correcting_steps(Func2d())
print(point.array)
print(Func2d()(point))
df.to_csv('my_data2.csv', index=False)

[-1.03718272  2.96957477]
13.117469869415531


## Лабораторная 12

In [24]:
def grad_method(func: Func, start_point: Point, step: float = 0.01, eps: float = 0.01) -> Point:
    
    grad = func.get_grad(start_point)
    new_point = func.create_point(start_point.array - step*grad.norm().array)
    while grad.get_mod() > eps:
        start_point = new_point
        new_point = func.create_point(new_point.array - step*grad.norm().array)
        grad = func.get_grad(start_point)
        
    return new_point
        

In [25]:
point = grad_method(Func2d(), Point2d([1,1]))
print(point.array)
print(Func2d()(point))

[0.42652636 0.70936133]
0.4285939720202552


In [26]:
def grad_method(func: Func, start_point: Point, step: float = 0.01, eps: float = 0.01) -> tuple[Point, pd.DataFrame]:
    
    df = []
    df.append(start_point.array)
    grad = func.get_grad(start_point)
    new_point = func.create_point(start_point.array - step*grad.norm().array)
    while grad.get_mod() > eps:
        start_point = new_point
        df.append(start_point.array)
        new_point = func.create_point(new_point.array - step*grad.norm().array)
        grad = func.get_grad(start_point)
        
    return new_point, pd.DataFrame(df)
        

In [27]:
point, df = grad_method(Func2d(), Point2d([1,1]))
print(point.array)
print(Func2d()(point))
df.to_csv('my_data3.csv', index=False)

[0.42652636 0.70936133]
0.4285939720202552


In [28]:
def grad_method_correcting_step(func: Func, start_point: Point, step: float = 1, eps: float = 0.01) -> Point:
    
    f_start_point = func(start_point)
    grad = func.get_grad(start_point)
    new_point = func.create_point(start_point.array - step*grad.norm().array)
    f_new_point = func(new_point)
    while grad.get_mod() > eps:
        start_point = new_point
        f_start_point = f_new_point
        new_point = func.create_point(new_point.array - step*grad.norm().array)
        f_new_point = func(new_point)
        grad = func.get_grad(start_point)
        
        if f_start_point < f_new_point:
            step /= 3
        
    return new_point

In [29]:
point = grad_method_correcting_step(Func2d(), Point2d([1,1]))
print(point.array)
print(Func2d()(point))

[0.4408601  0.70945094]
0.42895623940143013


In [41]:
def grad_method_correcting_step(func: Func, start_point: Point, max_iter: int = 1000,
                                step: float = 1, eps: float = 0.01) -> tuple[Point, pd.DataFrame]:
    
    
    
    df = []
    df.append(start_point.array)
    f_start_point = func(start_point)
    grad = func.get_grad(start_point)
    new_point = func.create_point(start_point.array - step*grad.norm().array)
    f_new_point = func(new_point)
    count_iter = 0
    while grad.get_mod() > eps and count_iter <= max_iter:
        count_iter += 1
        start_point = new_point
        df.append(start_point.array)
        f_start_point = f_new_point
        new_point = func.create_point(new_point.array - step*grad.norm().array)
        f_new_point = func(new_point)
        grad = func.get_grad(start_point)
        
        if f_start_point < f_new_point:
            step /= 3
        
    print(f"Глобальный минимум не был найден за {max_iter} итераций")
    return new_point, pd.DataFrame(df)

In [31]:
point, df = grad_method_correcting_step(Func2d(), Point2d([1,1]))
print(point.array)
print(Func2d()(point))
df.to_csv('my_data4.csv', index=False)

[0.4408601  0.70945094]
0.42895623940143013


In [32]:
class Func2dv2(Func):
    def get_grad(self, point: Point2d) -> Point2d:
        x1, x2 = point[0], point[1]
        return -Point2d([-2*x1+8, -2*x2+10])
    
    def create_point(self, arr: Sequence) -> Point7d:
        return Point2d(arr)
    
    def size(self) -> int:
        return 2
    
    def __call__(self, point: Point2d) -> float:
        x1, x2 = point[0], point[1]
        return -(5-(x1 - 4)**2 - (x2-5)**2)

In [33]:
point, df = grad_method_correcting_step(Func2dv2(), Point2d([0,0]))
print(point.array)
print(Func2dv2()(point))

[4.01038796 5.01298495]
-4.999723481381371


In [34]:
point, df = grad_method(Func2dv2(), Point2d([1,1]))
print(point.array)
print(Func2dv2()(point))

[4.006 5.008]
-4.9999


In [35]:
class Func2dv3(Func):
    
    
    def get_grad(self, point: Point2d) -> Point2d:
        x1, x2 = point[0], point[1]
        return -Point2d([4*x1-x2-1, 2*x2 - x1 - 1])
    
    def create_point(self, arr: Sequence) -> Point7d:
        return Point2d(arr)
    
    def size(self) -> int:
        return 2
    
    def __call__(self, point: Point2d) -> float:
        x1, x2 = point[0], point[1]
        return -(2*x1**2 - x1*x2 + x2**2 - x1 - x2 + 1)

In [42]:
point, df = grad_method_correcting_step(Func2dv3(), Point2d([1,1]))
print(point.array)
print(Func2dv3()(point))

Глобальный минимум не был найден за 1000 итераций
[ 931.18203695 -370.77985506]
-2216381.8113007136
